In [2]:
pip install neattext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext as nfx
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,GlobalMaxPooling1D,Input
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm




In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd

data = pd.read_csv("/content/drive/My Drive/Suicide_Detection.csv")

In [6]:
data['class'].value_counts()

suicide        116037
non-suicide    116037
Name: class, dtype: int64

In [7]:
data['class'].value_counts().index.values


array(['suicide', 'non-suicide'], dtype=object)

In [8]:
train_data,test_data =train_test_split(data,test_size =0.2,random_state =10)

In [9]:
train_data['class'].value_counts().index.values

array(['suicide', 'non-suicide'], dtype=object)

In [10]:
plx.bar(train_data,x=train_data['class'].value_counts().index.values,
        y=train_data['class'].value_counts(),color=['Suicide','Not Sucide'])

In [11]:
def clean_text(text):
  text_length =[]
  cleaned_text =[]
  for sent in tqdm(text):
    sent =sent.lower()
    sent =nfx.remove_special_characters(sent)
    sent =nfx.remove_stopwords(sent)
    text_length.append(len(sent.split()))
    cleaned_text.append(sent)
  return cleaned_text,text_length

In [12]:
cleaned_train_text,train_text_length=clean_text(train_data.text)
cleaned_test_text,test_text_length=clean_text(test_data.text)

100%|██████████| 46415/46415 [00:02<00:00, 16887.70it/s]


In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_train_text)

In [14]:
train_text_seq =tokenizer.texts_to_sequences(cleaned_train_text)
train_text_pad = pad_sequences(train_text_seq,maxlen=50)

test_text_seq =tokenizer.texts_to_sequences(cleaned_test_text)
test_text_pad =pad_sequences(test_text_seq,maxlen=50)


In [ ]:
train_text_seq

In [ ]:
train_text_pad

globe

In [17]:
lbl_target = LabelEncoder()
train_output = lbl_target.fit_transform(train_data['class'])
test_output = lbl_target.transform(test_data['class'])

In [18]:
import pickle

In [19]:
with open("/content/drive/My Drive/glove.840B.300d.pkl", "rb") as fp:
    glove_embedding = pickle.load(fp)

In [20]:
v=len(tokenizer.word_index)
embedding_matrix=np.zeros((v+1,300),dtype=float)
for word,idx in tokenizer.word_index.items():
  embedding_vector = glove_embedding.get(word)
  if embedding_vector is not None:
    embedding_matrix[idx]=embedding_vector

In [21]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.074482  ,  0.58293003, -0.78233999, ..., -0.24984001,
        -0.096953  ,  0.66692001],
       [-0.35394999,  0.23051   , -0.62689   , ..., -0.20720001,
         0.52003002,  0.51129001],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.29547   , -0.21822999, -0.039817  , ...,  0.62642998,
         0.48798001, -0.47554001],
       [ 0.75085002, -0.35099   ,  0.37674999, ..., -0.066863  ,
         0.79632998, -0.05967   ]])

In [22]:
early_stop =EarlyStopping(patience=5)
reducelr =ReduceLROnPlateau(patience=3)

In [23]:
model = Sequential()
model.add(Input(shape=(50,)))
model.add(Embedding(v+1,300,weights=[embedding_matrix],trainable=False))
model.add(LSTM(20,return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(256,activation ='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer =keras.optimizers.SGD(learning_rate=0.1,momentum=0.09),loss = 'binary_crossentropy',metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 300)           81560700  
                                                                 
 lstm (LSTM)                 (None, 50, 20)            25680     
                                                                 
 global_max_pooling1d (Glob  (None, 20)                0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 256)               5376      
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 81592013 (311.25 MB)
Trainable params: 31313 (122.32 KB)
Non-trainable params: 81560700 (311.13 MB)
________

In [25]:
train_text_pad.shape, train_output.shape, test_text_pad.shape, test_output.shape

((185659, 50), (185659,), (46415, 50), (46415,))

In [26]:
r = model.fit(train_text_pad, train_output, validation_data=(test_text_pad, test_output),
              epochs=20, batch_size=256, callbacks=[early_stop, reducelr])

Epoch 1/20
726/726 [==============================] - 58s 78ms/step - loss: 0.3453 - accuracy: 0.8536 - val_loss: 0.2575 - val_accuracy: 0.8981 - lr: 0.1000
Epoch 2/20
726/726 [==============================] - 56s 77ms/step - loss: 0.2462 - accuracy: 0.9013 - val_loss: 0.2264 - val_accuracy: 0.9111 - lr: 0.1000
Epoch 3/20
726/726 [==============================] - 61s 84ms/step - loss: 0.2249 - accuracy: 0.9109 - val_loss: 0.2610 - val_accuracy: 0.8954 - lr: 0.1000
Epoch 4/20
726/726 [==============================] - 69s 95ms/step - loss: 0.2077 - accuracy: 0.9185 - val_loss: 0.1999 - val_accuracy: 0.9214 - lr: 0.1000
Epoch 5/20
726/726 [==============================] - 56s 78ms/step - loss: 0.1906 - accuracy: 0.9257 - val_loss: 0.2245 - val_accuracy: 0.9123 - lr: 0.1000
Epoch 6/20
726/726 [==============================] - 56s 78ms/step - loss: 0.1789 - accuracy: 0.9305 - val_loss: 0.2012 - val_accuracy: 0.9208 - lr: 0.1000
Epoch 7/20
726/726 [==============================] - 57s 

In [27]:
!pip install scikit-learn

In [28]:
from sklearn.metrics import classification_report

In [29]:
print(type(model))

<class 'keras.src.engine.sequential.Sequential'>


In [30]:
print(test_output.dtype)
print(model.predict(test_text_pad).dtype)

print(train_output.dtype)
print(model.predict(train_text_pad).dtype)

int64
1451/1451 [==============================] - 14s 10ms/step
float32
int64
5802/5802 [==============================] - 53s 9ms/step
float32


In [31]:
  twt = ['i am happy']
  twt = tokenizer.texts_to_sequences(twt)
  twt = pad_sequences(twt,maxlen =50)

  prediction = model.predict(twt)[0][0]
  print (prediction)

  if(prediction  >0.5):
    print ("Potential Suicide Post")
  else:
    print("Non Suicide Post")


1/1 [==============================] - 0s 23ms/step
0.22004949
Non Suicide Post


In [32]:
pickle.dump(tokenizer, open('tokenizer.pkl','wb'))

In [33]:
model.save("model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [34]:
model.save('my_model.keras')

In [35]:
token_form = pickle.load(open('tokenizer.pkl','rb'))

In [36]:
from keras.models import load_model

In [37]:
model_form = load_model("model.h5")

In [38]:
twt = ['i am sad ']
twt = token_form.texts_to_sequences(twt)
twt = pad_sequences(twt,maxlen =50)


prediction = model_form.predict(twt)[0][0]
print(prediction)


if(prediction  >0.5):
    print ("Potential Suicide Post")
elif(prediction ==1):
    print("Non Suicide Post")


1/1 [==============================] - 1s 598ms/step
0.36623892


In [39]:
tf.keras.models.save_model(model,'my_mofrl2.hdf5')

<ipython-input-39-33f46f699bff>:1: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [40]:
model.save("/content/abc/model.h5")


In [46]:
file_path = '/content/app.py'


In [47]:
! pip install streamlit -q

In [48]:
import requests

response = requests.get('http://ipv4.icanhazip.com')
if response.status_code == 200:
    print("Your public IPv4 address is:", response.text.strip())
else:
    print("Failed to retrieve the IPv4 address.")


Your public IPv4 address is: 35.245.193.11


In [49]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.193.11:8501

npx: installed 22 in 3.113s
your url is: https://small-views-nail.loca.lt
2024-04-03 06:16:31.015433: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 06:16:31.015493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 06:16:31.017112: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 06:16:32.277716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-03 06:16:35.358749